# feature_selection

A la hora de trabajar con conjuntos de datos (datasets) que tienen un número grande de variables (features), puede ser muy útil aplicar técnicas de feature selection (selección de características). Estas técnicas nos ayudan a reducir la dimensionalidad del problema y, muchas veces, a mejorar el rendimiento de los modelos y su interpretabilidad.

En Scikit existen clases y métodos para esto en el módulo ``sklearn.model_selection``:

* ``VarianceThreshold``: 
    * El selector más sencillo. Elimina todas las características que no alcanzan un cierto nivel de varianza. Si una variable tiene la misma constante para todas las muestras (varianza = 0), no proporciona información útil.

* Selección univariante: Basadas en pruebas estadísticas univariadas entre cada característica y la variable objetivo (label). Es decir, se estudia cada feature por separado para determinar su relevancia.
    * Clases:
        * ``SelectKBest``: Selecciona el top k de características con mejor puntuación.
        * ``SelectPercentile``: Selecciona un porcentaje de características con mejor puntuación.
    * Métodos:
        * ``f_classif``: Test ANOVA F (para clasificación). Estima si cada feature ayuda a distinguir entre distintas clases analizando la diferencia de medias (ANOVA F) de la feature en cada clase.
        * ``chi2``: Test Chi-cuadrado (para clasificación con datos no negativos). Mide la (in)dependencia entre la feature y la clase calculando un estadístico $ χ2 $
        * ``f_regression``: Correlación lineal F (para regresión): Mide la correlación lineal (coeficiente de Pearson) y devuelve un estadístico F que indica qué tan fuerte es la relación lineal entre cada feature y la salida.
    
* ``RFE`` y ``RFECV``: Se entrena un estimador (por ejemplo, un modelo lineal o un árbol) con todas las características, se observa la importancia (o magnitud de los coeficientes) y se elimina la característica menos importante. Se repite el proceso recursivamente hasta llegar a un número de características deseado.

* ``SelectFromModel``:
    * Usa un modelo subyacente (generalmente lineal o basado en árboles) para estimar la importancia de cada característica, y luego selecciona aquellas con mayor relevancia. Funcionamiento:
        * Se entrena un estimador (por ejemplo, un modelo de regresión lineal con regularización L1, o un RandomForest).
        * Se miran los coeficientes (para modelos lineales) o las importancias de las características (para modelos de árboles).
        * Se aplica un threshold para descartar las que no superen la importancia mínima.


In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import f_classif, chi2, f_regression
from sklearn.feature_selection import VarianceThreshold, SelectKBest, SelectFromModel, RFE, RFECV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [7]:
data = load_breast_cancer() #clasificacion binaria
df = pd.DataFrame(data.data, columns= data.feature_names)
X = df
y = data.target

print(df.shape)
print(df.columns.to_list())

(569, 30)
['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension']


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

model = LogisticRegression(max_iter=5000)
model.fit(X_train, y_train)

print('accuracy train: ', model.score(X_train, y_train))
print('accuracy test: ', model.score(X_test, y_test))

accuracy train:  0.9626373626373627
accuracy test:  0.956140350877193


## VarianceThreshold

``VarianceThreshold`` es un método de selección de características basado en la varianza. Su función es eliminar aquellas características que no varían (o varían muy poco) a lo largo de las muestras.

La idea subyacente es que si una característica tiene muy poca variabilidad (por ejemplo, es constante o casi constante), es poco probable que aporte información discriminatoria o predictiva para el modelo.

Se define un umbral (por defecto, threshold=0), y se eliminan todas las características cuya varianza sea menor o igual a ese umbral.

No se formula una hipótesis estadística en el sentido clásico (H₀, H₁), pero se parte de la suposición de que las características con baja varianza son irrelevantes para el aprendizaje.

* Varianza 0: todos los valores de la característica con iguales (sin apenas variación)
* Varianza baja: valores muy cerca de la media y poca dispersión.
* Varianza alta: valores muy dispersos alrededor de la media.

In [27]:
selector = VarianceThreshold(threshold=0.2)
X_train_select = selector.fit_transform(X_train)
X_test_select = selector.transform(X_test)
print('columnas originales', X_train.shape[1])
print('columnas filtradas', X_train_select.shape[1])
selected_columns = X_train.columns[selector.get_support()]
print ('columnas seleccionadas ', selected_columns.to_list())

columnas originales 30
columnas filtradas 11
columnas seleccionadas  ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'texture error', 'perimeter error', 'area error', 'worst radius', 'worst texture', 'worst perimeter', 'worst area']


In [28]:
model = LogisticRegression(max_iter=5000)
model.fit(X_train_select, y_train)

print('accuracy train: ', model.score(X_train_select, y_train))
print('accuracy test: ', model.score(X_test_select, y_test))

accuracy train:  0.9582417582417583
accuracy test:  0.956140350877193


## selectKBest + f_classif

La función **`f_classif`** implementa la **prueba F de análisis de varianza (ANOVA)** para problemas de clasificación. 

Para cada característica, compara la varianza **entre los grupos** (definidos por las clases de la variable objetivo) con la varianza **dentro de cada grupo**. 

La idea es que si la media de la característica varía significativamente entre clases (entre-grupos) en comparación con la variabilidad interna (dentro de cada clase), esa característica es potencialmente informativa para la clasificación.


- **ANOVA F-test:**  
  Se basa en el cálculo de la estadística F, definida como:
  $$
  F = \frac{\text{MSB}}{\text{MSW}}
  $$
  donde:
  - **MSB (Mean Square Between):** Mide la variabilidad entre las medias de los diferentes grupos (clases).
  - **MSW (Mean Square Within):** Mide la variabilidad dentro de cada grupo.

Un valor alto de F sugiere que la característica tiene diferencias significativas entre clases, lo que indica que es útil para discriminar entre ellas. Además, se calculan **p-valores** que ayudan a evaluar la significación estadística del resultado.

Información relevante de SelectKBest:

* ``scores_``: Array con el valor de la estadística del test para cada característica. Valores altos indican mayor capacidad para discriminar (en clasificación) o para explicar la varianza (en regresión).

* ``pvalues_``: Es un array con el valor-p asociado a cada característica. El valor-p indica la probabilidad de observar, bajo la hipótesis nula (por ejemplo, que la característica no tenga relación con el target), un estadístico tan extremo o más extremo que el calculado. Interpretación:
    * Valores muy bajos (por ejemplo, < 0.05) sugieren que es poco probable que la relación observada se deba al azar, lo que indica que la característica es estadísticamente significativa.
    * Valores altos indican poca evidencia de relevancia.

- **H₀ (Hipótesis nula):** Las medias de la característica en cada grupo (clase) son iguales.  
  _Interpretación:_ La característica **no** discrimina entre las clases, es decir, no existe diferencia significativa en la media de la característica entre las distintas clases.

- **H₁ (Hipótesis alternativa):** Al menos una de las medias de la característica difiere de las otras.  
  _Interpretación:_ La característica **sí** discrimina entre las clases, lo que indica que existe una diferencia significativa en las medias y que la característica puede ser relevante para la clasificación.

Cuando observamos un **score alto** y un **valor-p muy bajo** para una característica, la interpretación es que se rechaza la hipótesis nula (H₀) en favor de la hipótesis alternativa (H₁). Es decir, se tiene evidencia estadística de que la característica es relevante para explicar la variabilidad en el target (ya sea en términos de discriminación entre clases o de relación lineal).

* ``get_support()``: Una vez ajustado el selector, podemos saber cuáles de las columnas originales han sido retenidas usando el método get_support(). Este método devuelve una máscara booleana del tamaño del número de características originales, en la que True indica que la característica fue seleccionada.

Nota: f_classif es el valor que usa por defecto SelectKBest si no indicamos nada.

In [34]:
selector = SelectKBest(score_func=f_classif) #k determina el nro de columnas que deseamos seleccionar que sean las mejores, por defecto tiene 10 se puede ampliar poniendo k=15 por ej
X_train_select = selector.fit_transform(X_train, y_train)
X_test_select = selector.transform(X_test)
print('columnas originales', X_train.shape[1])
print('columnas filtradas', X_train_select.shape[1])
selected_columns = X_train.columns[selector.get_support()]
print ('columnas seleccionadas ', selected_columns.to_list())
print('scores: ', selector.scores_[:5]) #ver los 5 primeros scores
print('p_values: ', selector.pvalues_[:5])


columnas originales 30
columnas filtradas 10
columnas seleccionadas  ['mean radius', 'mean perimeter', 'mean area', 'mean concavity', 'mean concave points', 'worst radius', 'worst perimeter', 'worst area', 'worst concavity', 'worst concave points']
scores:  [482.2339446   94.91778777 522.4892668  423.65413321  74.1901474 ]
p_values:  [2.56681180e-73 1.72622610e-20 1.80197085e-77 6.11197054e-67
 1.18705059e-16]


In [35]:
model = LogisticRegression(max_iter=5000)
model.fit(X_train_select, y_train)

print('accuracy train: ', model.score(X_train_select, y_train))
print('accuracy test: ', model.score(X_test_select, y_test))

accuracy train:  0.9406593406593406
accuracy test:  0.9912280701754386


## SelectKBest + chi2

Va mas para conteos o frecuencias (nro de clientes visitados al dia, cantidad de veces que ocurre algo)

La función **`chi2`** aplica la **prueba de chi-cuadrado** para evaluar la dependencia entre cada característica de la "X" y la variable objetivo "y" en problemas de clasificación. 

Esta prueba es especialmente útil cuando se trabaja con características que representan **frecuencias o conteos** (por ejemplo, en problemas de procesamiento de lenguaje natural con datos de conteo de palabras).

- **Chi-cuadrado (χ²):**  
  Se utiliza para comparar las frecuencias observadas en cada categoría con las frecuencias esperadas bajo la hipótesis nula de independencia entre la característica y la variable objetivo. La estadística se calcula mediante:
  $$
  \chi^2 = \sum \frac{(O_i - E_i)^2}{E_i}
  $$
  donde:
  - $ O_i $ es la frecuencia observada.
  - $ E_i $ es la frecuencia esperada si las variables fueran independientes.

  Un valor elevado de χ² indica que hay una relación significativa entre la característica y la clase, lo que sugiere que la característica es informativa.  
  
  **Importante:** Para usar `chi2` las características deben ser **no negativas**, ya que representan conteos o frecuencias.

In [39]:
selector = SelectKBest(score_func=chi2) #k determina el nro de columnas que deseamos seleccionar que sean las mejores, por defecto tiene 10 se puede ampliar poniendo k=15 por ej
X_train_select = selector.fit_transform(X_train, y_train)
X_test_select = selector.transform(X_test)
print('columnas originales', X_train.shape[1])
print('columnas filtradas', X_train_select.shape[1])
selected_columns = X_train.columns[selector.get_support()]
print ('columnas seleccionadas ', selected_columns.to_list())
print('scores: ', selector.scores_[:5]) #ver los 5 primeros scores
print('p_values: ', selector.pvalues_[:5])


columnas originales 30
columnas filtradas 10
columnas seleccionadas  ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'perimeter error', 'area error', 'worst radius', 'worst texture', 'worst perimeter', 'worst area']
scores:  [2.07305367e+02 7.46050345e+01 1.56559227e+03 4.22404263e+04
 1.29356778e-01]
p_values:  [5.31833982e-47 5.74963942e-18 0.00000000e+00 0.00000000e+00
 7.19099888e-01]


In [40]:
model = LogisticRegression(max_iter=5000)
model.fit(X_train_select, y_train)

print('accuracy train: ', model.score(X_train_select, y_train))
print('accuracy test: ', model.score(X_test_select, y_test))

accuracy train:  0.945054945054945
accuracy test:  0.9649122807017544


## SelectKBest + f_regresion

La función `f_regression` está diseñada para problemas de **regresión**. 

Evalúa la relación lineal entre cada característica y una variable objetivo continua. Para cada característica, se calcula un estadístico F derivado de la correlación (usualmente la correlación de Pearson) entre la característica y el target.

- **F-test para regresión:**  
  Se parte de la hipótesis de que existe una relación lineal entre la característica y la variable dependiente. Se calcula el coeficiente de correlación $ r $ y se transforma en una estadística F:
  $$
  F = \frac{r^2}{1 - r^2} \times \frac{n - 2}{1}
  $$
  donde $ n $ es el número de muestras.  
  
Un valor alto de F sugiere que la característica tiene una relación lineal fuerte con la variable objetivo, y por tanto es relevante para la predicción en tareas de regresión.

In [41]:
selector = SelectKBest(score_func=f_regression) #k determina el nro de columnas que deseamos seleccionar que sean las mejores, por defecto tiene 10 se puede ampliar poniendo k=15 por ej
X_train_select = selector.fit_transform(X_train, y_train)
X_test_select = selector.transform(X_test)
print('columnas originales', X_train.shape[1])
print('columnas filtradas', X_train_select.shape[1])
selected_columns = X_train.columns[selector.get_support()]
print ('columnas seleccionadas ', selected_columns.to_list())
print('scores: ', selector.scores_[:5]) #ver los 5 primeros scores
print('p_values: ', selector.pvalues_[:5])

columnas originales 30
columnas filtradas 10
columnas seleccionadas  ['mean radius', 'mean perimeter', 'mean area', 'mean concavity', 'mean concave points', 'worst radius', 'worst perimeter', 'worst area', 'worst concavity', 'worst concave points']
scores:  [482.2339446   94.91778777 522.4892668  423.65413321  74.1901474 ]
p_values:  [2.56681180e-73 1.72622610e-20 1.80197085e-77 6.11197054e-67
 1.18705059e-16]


In [42]:
model = LogisticRegression(max_iter=5000)
model.fit(X_train_select, y_train)

print('accuracy train: ', model.score(X_train_select, y_train))
print('accuracy test: ', model.score(X_test_select, y_test))

accuracy train:  0.9406593406593406
accuracy test:  0.9912280701754386


## RFE y RFECV